In [1]:
!pip install --upgrade openai numpy pandas scikit-learn tiktoken
!pip install faiss-cpu tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.8/767.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.9.0
    Uninstalling tiktoken-0.9.0:
      Successfully uninstalled tiktoken-0.9.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.8 MB/s eta 0:00:00
^C


In [1]:
# upload FAQ file
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd

# fetch the file name
filename = list(uploaded.keys())[0]

# read the .CSV as DataFrame
faq_df = pd.read_csv(io.BytesIO(uploaded[filename]))

# show the top 5 row
print("✅ The file uploaded and read successfully：")
faq_df.head()

Saving cleaned_faq.csv to cleaned_faq.csv
✅ The file uploaded and read successfully：


,Question,Answer
0,What is Mindful Ocean Metaverse?,Mindful Ocean Metaverse is a community art pro...
1,How can we save our ocean by making art?,We believe art-making can cultivate mindfulnes...
2,What is United Nations' Sustainable Developmen...,The United Nations' Sustainable Development Go...
3,What is UNSDG #14 Life Below Water?,"""Life Below Water"" is one of the 17 Sustainabl..."
4,Why should I care about the ocean?,Caring about the ocean is essential because it...


In [2]:
# remove the useless spaces and show the name of all column
faq_df.columns = faq_df.columns.str.strip()
print("📌 Column name：", faq_df.columns.tolist())

# the most-related columns in FAQ
question_col = "Question"
answer_col = "Answer"

# remove the null value
faq_df = faq_df.dropna(subset=[question_col, answer_col])

# remove the useless spaces
faq_df[question_col] = faq_df[question_col].str.strip()
faq_df[answer_col] = faq_df[answer_col].str.strip()

# show the top 5 rows of the cleaned data
print(f"✅ total rows of the cleaned data：{len(faq_df)}")
faq_df.head()

📌 Column name： ['Question', 'Answer']
✅ total rows of the cleaned data：89


,Question,Answer
0,What is Mindful Ocean Metaverse?,Mindful Ocean Metaverse is a community art pro...
1,How can we save our ocean by making art?,We believe art-making can cultivate mindfulnes...
2,What is United Nations' Sustainable Developmen...,The United Nations' Sustainable Development Go...
3,What is UNSDG #14 Life Below Water?,"""Life Below Water"" is one of the 17 Sustainabl..."
4,Why should I care about the ocean?,Caring about the ocean is essential because it...


In [6]:
from openai import OpenAI
from google.colab import userdata
import numpy as np

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# get all the questions
questions = faq_df[question_col].tolist()

# get the vector
print("🔄 generating embeddings...")

embeddings = []
for q in questions:
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=q
    )
    embeddings.append(response.data[0].embedding)

# add the vector back to DataFrame
faq_df["embedding"] = embeddings
print("✅ Embeddings genereated！")

🔄 generating embeddings...
✅ Embeddings genereated！


In [7]:
import faiss

# convert embedding to float32 numpy array
embedding_matrix = np.array(faq_df["embedding"].to_list()).astype("float32")

# create FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

print(f"✅ FAISS index built with {index.ntotal} entries.")

✅ FAISS index built with 89 entries.


In [8]:
def query_with_context(user_query, top_k=3):
    # convert questions to vector
    query_embedding = client.embeddings.create(
        model="text-embedding-3-small",
        input=user_query
    ).data[0].embedding

    # similar top-k questions query
    query_vector = np.array([query_embedding]).astype("float32")
    _, indices = index.search(query_vector, top_k)

    # fetch the most related Q&A
    context = ""
    for i in indices[0]:
        q = faq_df.iloc[i][question_col]
        a = faq_df.iloc[i][answer_col]
        context += f"Q: {q}\nA: {a}\n\n"

    # send the context and questions to GPT
    final_prompt = f"""You are a helpful assistant. Here are some related FAQs:

{context}
Now based on the above FAQs, answer the following user question clearly:

"{user_query}"
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0.3
    )

    return response.choices[0].message.content

In [9]:
user_question = "How do I get more OC points?"
print("🤖 AI Answer:\n")
print(query_with_context(user_question))

🤖 AI Answer:

The FAQs provided do not specifically mention how to earn OC points. However, if OC points are related to the activities mentioned, you might consider engaging with the Mindful Ocean platform by creating artwork, participating in the Mint Pass process, or collaborating with green organizations. For more detailed information on earning OC points, please refer to the official guidelines or support resources of the Mindful Ocean platform.
